In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [3]:
# import torch
# torch.ones(1,2).cuda()

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

K = 2
STITCH_BATCH_SIZE = 32 # todo study the effect
MAX_DEPTH = 16
THRESOULD = 0
TOTAL_THRESOULD = 0.5

RESULT_NAME = f"{int(time.time())}_result_R2_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}_K_{K}"

EVAL_BATCH_SIZE = 16

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, batch_size=STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [5]:
data_score.shape

(32, 3, 224, 224)

In [6]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [7]:
# import torch
# x = torch.ones(1024, 120)
# W = torch.ones(48, 120)
# # (1024x120 and 48x120)
# x.matmul(W.T).shape

In [11]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score, scoring_method='R2')
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False, K=K)
    for i,(s,net) in enumerate(generator):
        try:
            netname = f"_results/{RESULT_NAME}/net{k:03}"
            report.evaluate(nets, net, netname, s, dataset_val)
            net.save(netname)
            k += 1
        except Exception as e:
            print('ERROR', e)
            pass

current depth: 1
r2 0.6151289939880371r2 0.9999998807907104 150192.296875 150192.3125
r2 0.7533265352249146 103948.4453125 137985.90625
 88900.296875 144523.015625r2 0.8823055028915405 131460.734375 148996.84375

r2 0.6643925309181213 20790.5625 31292.587890625
r2 0.8678284287452698 114814.4453125 132300.859375
r2 0.8721227049827576 117299.140625 134498.4375
r2 0.9659315943717957 137661.703125 142517.03125
r2 0.6667889952659607 108840.7109375 163231.109375
r2 0.5023900866508484 18186.939453125 36200.83203125
r2 0.9690576195716858 134332.890625 138622.1875
r2 0.9111186265945435 122707.859375 134678.25
X r2r2 0.7953624129295349 114423.46875 143863.3125
 0.7454679012298584 113422.1875 152148.984375
tensor([[0.0000, 0.0000, 0.6047,  ..., 0.0000, 7.3086, 0.1839],
        [0.0000, 0.0000, 1.1116,  ..., 0.0000, 0.0000, 0.8449],
        [0.2196, 0.2525, 0.8664,  ..., 0.8594, 3.4555, 0.5092],
        ...,
        [0.5532, 0.0000, 0.0000,  ..., 0.8621, 0.0000, 1.3015],
        [0.0000, 0.0000, 0

100%|██████████| 80/80 [00:25<00:00,  3.08it/s]


accuracy 0.4819253438113949
saving to _results/1689182147_result_R2_BS_32_MD_16_T_0_TT_0.5_K_5/net009
curr <stitchnet.stitchonnx.utils.Fragment object at 0x7f9e9d3966e0>
nextf <stitchnet.stitchonnx.utils.Fragment object at 0x7f9dc49945b0>
totalscore 1.0000001192092896
epoch 0 loss 0.0 torch.Size([32, 64]) torch.Size([32, 1024])


100%|██████████| 80/80 [00:25<00:00,  3.18it/s]


accuracy 0.4548133595284872
saving to _results/1689182147_result_R2_BS_32_MD_16_T_0_TT_0.5_K_5/net010
curr <stitchnet.stitchonnx.utils.Fragment object at 0x7f9e9d3966e0>
nextf <stitchnet.stitchonnx.utils.Fragment object at 0x7f9dc4995cc0>
totalscore 1.0000001192092896
epoch 0 loss 0.0 torch.Size([32, 64]) torch.Size([32, 25088])
(32, 3, 224, 224)
current depth: 2
r2 0.9999998211860657 23338.99609375 23339.0
X tensor([[0.0000, 0.0000, 2.2965,  ..., 0.0000, 0.4828, 0.0000],
        [0.0000, 1.4921, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [2.7609, 0.0000, 2.0416,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 2.4225, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
Y tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1228],
        [0.0000, 0.1352, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1459, 0.0827,  .

...

Traceback (most recent call last):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1100, in recursive_stitching
    newcurr_fragment = stitch_fragments(curr, nextf, data)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 731, in stitch_fragments
    nw = adjust_w(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 639, in adjust_w
    raise Exception("unsupport linear to conv stitching")
Exception: unsupport linear to conv stitching


ERROR The size of tensor a (64) must match the size of tensor b (1024) at non-singleton dimension 1
curr <stitchnet.stitchonnx.utils.Fragment object at 0x7f9e9d3966e0>
nextf <stitchnet.stitchonnx.utils.Fragment object at 0x7f9dc4995210>
totalscore 1.0


Traceback (most recent call last):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1100, in recursive_stitching
    newcurr_fragment = stitch_fragments(curr, nextf, data)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 731, in stitch_fragments
    nw = adjust_w(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 641, in adjust_w
    return adjust_w_conv(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 617, in adjust_w_conv
    print('diff sampled', (acts1sampled - acts2sampled).pow(2).sum())
RuntimeError: The size of tensor a (64) must match the size of tensor b (1024) at non-singleton dimension 1


epoch 0 loss 0.0 torch.Size([32, 64]) torch.Size([32, 1024])


 56%|█████▋    | 45/80 [00:14<00:11,  3.12it/s]
Traceback (most recent call last):
  File "/tmp/ipykernel_1388191/2548392931.py", line 12, in <cell line: 4>
    report.evaluate(nets, net, netname, s, dataset_val)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/report.py", line 14, in evaluate
    accuracy = accuracy_score_net(net, dataset, bs=self.bs)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1318, in accuracy_score_net
    for x,t in tqdm(load_dl(dataset, batch_size=bs, shuffle=False, num_workers=num_workers)):
  File "/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 681, in __next__
    data = self._next_data()
  File "/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 72

In [ ]:
# dataset_val

In [ ]:
print('DONE')